In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

data = loadmat("ex4data1.mat")
weights = loadmat("ex3weights.mat")
print(data.keys())
print(weights.keys())

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])
dict_keys(['__header__', '__version__', '__globals__', 'Theta1', 'Theta2'])


In [ ]:
X = data['X']
y = data['y']
y = pd.get_dummies(y.ravel()).values
theta1_loaded = weights["Theta1"]
theta2_loaded = weights["Theta2"]
print(X.shape)
print(y.shape)
print(theta1_loaded.shape)

(5000, 400)
(5000, 10)
(25, 401)


In [ ]:
import numpy as np
import pandas as pd

from scipy.optimize import minimize
from time import time

class NeuralNetwork:
    """Creates Neural Network for MNIST dataset."""
    def __init__(self, hidden_size=25, output_size=10, lambda_=0):
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.thetas = None
        self.lambda_ = lambda_
        self.X = None
        self.y = None
        self.input_size = 0

    @staticmethod
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))

    @staticmethod
    def flatten(arr1, arr2):
        return np.r_[arr1.flatten(), arr2.flatten()]
    
    def set_params(self, *thetas):
        self.thetas = self.flatten(*thetas)

    def unflatten(self, arr):
        theta1 = arr[:self.hidden_size * (self.input_size + 1)]
        theta1 = theta1.reshape(self.hidden_size, self.input_size + 1)
        theta2 = arr[self.hidden_size * (self.input_size + 1):]
        theta2 = theta2.reshape(self.output_size, self.hidden_size + 1)
        return theta1, theta2

    def init_random_thetas(self, epsilon=0.12):
        theta1 = np.random.rand(self.hidden_size, self.input_size + 1) \
                 * 2 * epsilon - epsilon
        theta2 = np.random.rand(self.output_size, self.hidden_size + 1) \
                 * 2 * epsilon - epsilon
        return self.flatten(theta1, theta2)

    def sigmoid_prime(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def cross_entropy(self, thetas=None):
        if thetas is None:
            theta1, theta2 = self.unflatten(self.thetas)
        else:
            theta1, theta2 = self.unflatten(thetas)
        m = self.X.shape[0]
        y_pred = self.forward_pass(thetas)
        positive_loss = np.sum(np.multiply(self.y, np.log(y_pred)).flatten())
        negative_loss = np.sum(np.multiply((1 - self.y), np.log(1 - y_pred))
                               .flatten())
        regularization = (self.lambda_ / (2 * m)) \
                         * (np.sum(theta1.flatten() ** 2)
                            + np.sum(theta2.flatten() ** 2))
        J = - (1 / m) * (positive_loss + negative_loss) + regularization
        return J

    def forward_pass(self, thetas=None, elaborate=False):
        if thetas is None:
            theta1, theta2 = self.unflatten(self.thetas)
        else:
            theta1, theta2 = self.unflatten(thetas)
        a1 = np.c_[np.ones(self.X.shape[0]), self.X]
        z2 = theta1.dot(a1.T)                   # 25x401 * 401x5000 = 25x5000
        a2 = self.sigmoid(z2.T)                 # 5000x25
        a2 = np.c_[np.ones(a2.shape[0]), a2]    # 5000x26
        z3 = theta2.dot(a2.T)                   # 10x26 * 26x5000 = 10x5000
        a3 = self.sigmoid(z3.T)                 # 5000x10
        if elaborate:
            return (a1, a2, a3), (z2, z3)
        return a3

    def backward_pass(self, thetas=None):
        if thetas is None:
            theta1, theta2 = self.unflatten(self.thetas)
        else:
            theta1, theta2 = self.unflatten(thetas)
        (a1, a2, y_pred), (z2, z3) = self.forward_pass(thetas, elaborate=True)
        delta3 = np.multiply((y_pred - self.y), self.sigmoid_prime(z3.T))
        theta2_grad = a2.T.dot(delta3)
        theta2_grad = theta2_grad.T
        # theta2_grad.shape is now same as theta2.shape
        delta2 = np.multiply(delta3.dot(theta2[:, 1:]), self.sigmoid_prime(z2.T))
        theta1_grad = a1.T.dot(delta2)
        theta1_grad = theta1_grad.T
        return self.flatten(theta1_grad, theta2_grad)

    def gradient_descent(self, X, y, n_epochs=1000, alpha=0.001):
        self.thetas = self.init_random_thetas()
        theta1, theta2 = self.unflatten(self.thetas)
        
        for i in range(1, n_epochs+1):
            cost = self.cross_entropy()
            print("\nIteration: {0} Cost: {1}".format(i, cost), end="")
            theta1_grad, theta2_grad = self.unflatten(self.backward_pass())
            theta1 = theta1 - alpha * theta1_grad
            theta2 = theta2 - alpha * theta2_grad
            self.thetas = self.flatten(theta1, theta2)
        print()
        
    def fmin_unc(self, X, y, **params):
        self.thetas = self.init_random_thetas()
        res = minimize(self.cross_entropy, self.thetas, jac=self.backward_pass,
                       method="tnc", options=params)
        print(res)

    def fit(self, X, y):
        if y.shape[1] != self.output_size:
            raise ValueError("Number of columns in y ({0}) are != to number "
                             "of output neurons ({1})"
                             .format(y.shape[1], self.output_size))
        self.X = X
        self.y = y
        self.input_size = X.shape[1]
    
    def train(self, method="gradient_descent", **params):
        start_time = time()
        if method == "gradient_descent":
            self.gradient_descent(X, y, **params)
        else:
            self.fmin_unc(X, y, **params)
        print("Training time: {0:.2f} secs".format(time() - start_time))


In [ ]:
nn = NeuralNetwork(hidden_size=25, output_size=10)
nn.fit(X, y)
print("Training using Gradient Descent")
nn.train()

Training using Gradient Descent

Iteration: 1 Cost: 7.239800627115109
Iteration: 2 Cost: 3.7661739998438346
Iteration: 3 Cost: 3.6235027175309416
Iteration: 4 Cost: 3.495570320933034
Iteration: 5 Cost: 3.394714003737753
Iteration: 6 Cost: 3.3284495184481284
Iteration: 7 Cost: 3.2930626727631127
Iteration: 8 Cost: 3.2765942020545022
Iteration: 9 Cost: 3.268393818949133
Iteration: 10 Cost: 3.263100970066674
Iteration: 11 Cost: 3.258782313865413
Iteration: 12 Cost: 3.2548199968625897
Iteration: 13 Cost: 3.251002449276057
Iteration: 14 Cost: 3.2472407321943337
Iteration: 15 Cost: 3.243486771747217
Iteration: 16 Cost: 3.239708461029159
Iteration: 17 Cost: 3.235881093454855
Iteration: 18 Cost: 3.231983922819274
Iteration: 19 Cost: 3.2279985188423193
Iteration: 20 Cost: 3.2239078292445122
Iteration: 21 Cost: 3.219695558822508
Iteration: 22 Cost: 3.215345712892498
Iteration: 23 Cost: 3.2108422381563697
Iteration: 24 Cost: 3.2061687270816184
Iteration: 25 Cost: 3.2013081657727214
Iteration: 26 

In [ ]:
nn = NeuralNetwork(hidden_size=25, output_size=10)
nn.fit(X, y)
print("Training using Newton's Conjugate Gradient")
nn.train(method="newton", maxiter=1000)

Training using Newton's Conjugate Gradient
     fun: 0.24530967006753418
     jac: array([ 0.08023951,  0.        ,  0.        , ..., -0.01913908,
        0.08889889,  0.0210124 ])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 639
     nit: 36
  status: 1
 success: True
       x: array([ 0.67535798,  0.10883192,  0.08662833, ...,  7.85126826,
        1.323868  , -2.77529457])
Training time: 46.91 secs


In [ ]:
nn = NeuralNetwork(hidden_size=25, output_size=10)
nn.fit(X, y)
nn.set_params(theta1_loaded, theta2_loaded)
nn.cross_entropy()

0.28762916516131887